In [1]:
# Application of FL task
from MLModel import *
from FLModel import *
from utils import *

from torchvision import datasets, transforms
import torch
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

In [2]:
def load_cnn_mnist(num_users):
    data_train = datasets.MNIST(root="~/data/", train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

    data_test = datasets.MNIST(root="~/data/", train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

    # split MNIST (training set) into non-iid data sets
    non_iid = []
    user_dict = mnist_noniid(data_train, num_users)
    for i in range(num_users):
        idx = user_dict[i]
        d = data_train.data[idx].float().unsqueeze(1)
        targets = data_train.targets[idx].float()
        non_iid.append((d, targets))
    non_iid.append((data_test.data.float().unsqueeze(1), data_test.targets.float()))
    return non_iid


def load_mnist(num_users):
    data_train = datasets.MNIST(root="~/data/", train=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

    data_test = datasets.MNIST(root="~/data/", train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

    # split MNIST (training set) into non-iid data sets
    non_iid = []
    user_dict = mnist_noniid(data_train, num_users)
    for i in range(num_users):
        idx = user_dict[i]
        d = data_train.data[idx].flatten(1).float()
        targets = data_train.targets[idx].float()
        non_iid.append((d, targets))
    non_iid.append((data_test.data.flatten(1).float(), data_test.targets.float()))
    return non_iid

In [3]:
"""
1. load_data
2. generate clients (step 3)
3. generate aggregator
4. training
"""
client_num = 5
d = load_cnn_mnist(client_num)

In [4]:
"""
FL model parameters.
"""
lr = 0.1
fl_param = {
    'output_size': 10,
    'client_num': client_num,
    'model': MnistCNN,
    'data': d,
    'lr': lr,
    'E': 100,
    'C': 1,
    'eps': 8.0,
    'delta': 1e-5,
    'q': 0.05,
    'clip': 8,
    'tot_T': 10,
    'batch_size': 32,
    'device': device
}
import warnings
warnings.filterwarnings("ignore")
fl_entity = FLServer(fl_param).to(device)

DP-SGD with sampling rate = 5% and noise_multiplier = 4.5770032215774075 iterated over 20000 steps satisfies differential privacy with eps = 8 and delta = 1e-05.


In [5]:
acc = []
for t in range(fl_param['tot_T']):
    fl_entity.set_lr(lr/np.sqrt(t+1))
    acc += [fl_entity.global_update()]
    print("global epochs = {:d}, acc = {:.4f}".format(t+1, acc[-1]))

global epochs = 1, acc = 0.5503
global epochs = 2, acc = 0.8287
global epochs = 3, acc = 0.8812
global epochs = 4, acc = 0.9002
global epochs = 5, acc = 0.9032
global epochs = 6, acc = 0.9278
global epochs = 7, acc = 0.9294
global epochs = 8, acc = 0.9403
global epochs = 9, acc = 0.9456
global epochs = 10, acc = 0.9483
